In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [69]:
train_data= pd.read_csv('train_v9rqX0R.csv')
test_data= pd.read_csv('test_AbJTz2l.csv')
y= train_data.pop('Item_Outlet_Sales')

In [70]:
item_weight ={}
for i, j in zip(train_data.Item_Identifier,train_data.Item_Weight):
    if np.isnan(j):
        continue
    else:
        item_weight[i]=j

new_item_weight=[]
for i, j in zip(train_data.Item_Identifier,train_data.Item_Weight):
    if np.isnan(j):
        if i in item_weight:
            new_item_weight.append(item_weight[i])
        else:
            new_item_weight.append(j)
    else:
        new_item_weight.append(j)
train_data.Item_Weight=new_item_weight

new_item_weight=[]
for i, j in zip(test_data.Item_Identifier,test_data.Item_Weight):
    if np.isnan(j):
        if i in item_weight:
            new_item_weight.append(item_weight[i])
        else:
            new_item_weight.append(j)
    else:
        new_item_weight.append(j)
test_data.Item_Weight=new_item_weight

In [71]:
outlet_size =[]
for loc_type,size,outlet_type,year in zip(train_data.Outlet_Location_Type,train_data.Outlet_Size,train_data.Outlet_Type,train_data.Outlet_Establishment_Year):
    if type(size)==float:
        if outlet_type=='Supermarket Type3' or outlet_type=='Supermarket Type2':
            outlet_size.append('Medium')
            continue
        elif outlet_type=='Grocery Store':
            outlet_size.append('Small')
            continue
        elif year==1987:
            outlet_size.append('High')
            continue
        elif year==1997 or year==2004:
            outlet_size.append('Small')
            continue
        elif year==1999 or year==2009:
            outlet_size.append('Small')
            continue
        elif loc_type=='Tier 2':
            outlet_size.append('Small')
            continue 
        elif loc_type=='Tier 3':
            outlet_size.append('Medium')
            continue 
        elif loc_type=='Tier 1':
            outlet_size.append('Small')
            continue
    else:
        outlet_size.append(size)
train_data.Outlet_Size= outlet_size

In [72]:
outlet_size =[]
for loc_type,size,outlet_type,year in zip(test_data.Outlet_Location_Type,test_data.Outlet_Size,test_data.Outlet_Type,test_data.Outlet_Establishment_Year):
    if type(size)==float:
        if outlet_type=='Supermarket Type3' or outlet_type=='Supermarket Type2':
            outlet_size.append('Medium')
            continue
        elif outlet_type=='Grocery Store':
            outlet_size.append('Small')
            continue
        elif year==1987:
            outlet_size.append('High')
            continue
        elif year==1997 or year==2004:
            outlet_size.append('Small')
            continue
        elif year==1999 or year==2009:
            outlet_size.append('Small')
            continue
        elif loc_type=='Tier 2':
            outlet_size.append('Small')
            continue 
        elif loc_type=='Tier 3':
            outlet_size.append('Medium')
            continue 
        elif loc_type=='Tier 1':
            outlet_size.append('Small')
            continue
    else:
        outlet_size.append(size)
test_data.Outlet_Size= outlet_size

In [73]:
categorical_col= train_data.select_dtypes('object').columns
train_data.loc[:,categorical_col]= train_data.loc[:,categorical_col].astype('str')
test_data.loc[:,categorical_col]= test_data.loc[:,categorical_col].astype('str')

In [74]:
X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.2, random_state=42)

In [75]:
from catboost import CatBoostRegressor
import numpy as np

# initialize the model
cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6,\
                          loss_function='RMSE', cat_features=list(categorical_col),nan_mode='Min')

# fit the model on the training data
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), logging_level='Silent',plot=True)

# make predictions on the test set
y_pred = cat_model.predict(X_test)

# evaluate the model
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

RMSE:  1015.8409919753993


Let's see feature importance

In [76]:
pd.DataFrame(train_data.columns,cat_model.feature_importances_)

,0
0.892894,Item_Identifier
0.521689,Item_Weight
1.153518,Item_Fat_Content
0.477762,Item_Visibility
1.196706,Item_Type
51.879821,Item_MRP
19.465338,Outlet_Identifier
1.924649,Outlet_Establishment_Year
1.804300,Outlet_Size
1.479738,Outlet_Location_Type


In [77]:
prediction=cat_model.predict(test_data)

In [78]:
output = pd.DataFrame({'Item_Identifier': test_data.Item_Identifier,
                       'Outlet_Identifier': test_data.Outlet_Identifier,
                       'Item_Outlet_Sales': prediction})
output.to_csv('submission1.csv', index=False)

In [79]:
# submission error =  1150

In [80]:
new_data_frame= train_data.loc[:,['Item_MRP','Outlet_Identifier','Outlet_Type']]

In [81]:
categorical_col2= new_data_frame.select_dtypes('object').columns

In [82]:
X_train, X_test, y_train, y_test = train_test_split(new_data_frame, y, test_size=0.2, random_state=42)

In [83]:
from catboost import CatBoostRegressor
import numpy as np

# initialize the model
cat_model2 = CatBoostRegressor(iterations=1000, learning_rate=0.01, depth=6,\
                          loss_function='RMSE', cat_features=list(categorical_col2),nan_mode='Min')

# fit the model on the training data
cat_model2.fit(X_train, y_train, eval_set=(X_test, y_test), logging_level='Silent',plot=True)

# make predictions on the test set
y_pred = cat_model2.predict(X_test)

# evaluate the model
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: ", rmse)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

RMSE:  1017.475517465378


In [84]:
new_test_data= test_data.loc[:,['Item_MRP','Outlet_Identifier','Outlet_Type']]

In [85]:
prediction=cat_model2.predict(new_test_data)

In [86]:
output = pd.DataFrame({'Item_Identifier': test_data.Item_Identifier,
                       'Outlet_Identifier': test_data.Outlet_Identifier,
                       'Item_Outlet_Sales': prediction})
output.to_csv('submission2.csv', index=False)